In [ ]:
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data.builtin import USEquityPricing  
from quantopian.pipeline.data.sentdex import sentiment
from quantopian.pipeline.data.morningstar import operation_ratios
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.data import morningstar
import quantopian.pipeline.filters as Filters


import numpy as np

class High252(CustomFactor):  
    window_length = 252 # ~52 weeks  
    inputs = [USEquityPricing.close]  
    def compute(self, today, asset_ids, out, close_prices):  
        out[:] = np.max(close_prices, axis=0)

class Low252(CustomFactor):  
    window_length = 252 # ~52 weeks  
    inputs = [USEquityPricing.close]  
    def compute(self, today, asset_ids, out, close_prices):  
        out[:] = np.min(close_prices, axis=0)  
    
class fivedays(CustomFactor):
    window_length = 5
    inputs = [USEquityPricing.close]  
    def compute(self, today, asset_ids, out, close_prices):  
        out[:] = abs(close_prices[0]-close_prices[-1])/close_prices[0]
       
class SPY_Close_Price(CustomFactor):  

    inputs = [USEquityPricing.close]  
    window_length = 1  
    def compute(self, today, assets, out, close):

        spy_index = np.where(assets == 8554)   # 8554 is the SID of SPY  
        # index -1 gets the latest price row. spy_index gets the SPY column.  
        out[:] = close[-1, spy_index]  
        
class SPY_252(CustomFactor):  
    
    inputs = [USEquityPricing.close]  
    window_length = 252  
    def compute(self, today, assets, out, close):

        spy_index = np.where(assets == 8554)   # 8554 is the SID of SPY  
        # index -1 gets the latest price row. spy_index gets the SPY column.  
        out[:] = np.max(close[:,spy_index])

def make_pipeline():
    
    high = High252()/USEquityPricing.close.latest
    low = Low252()/USEquityPricing.close.latest
    
    market_cap= morningstar.valuation.market_cap.latest  

    universe = (high.notnull()
                & low.notnull()
                & (market_cap < 500000000)
                & (market_cap > 10000000))
    
    spyquantile = fivedays().percentile_between(0, 20, mask = universe)
    highquantile = high.percentile_between(90, 100, mask = universe)
    sma_20 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=20, mask = universe)
    sma_50 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=50, mask = universe)
    spy252 = (SPY_Close_Price()/SPY_252()) < 0.9
    
    
    pipe = Pipeline(columns={
            'longs': highquantile & (sma_20>sma_50) & spyquantile,
            'shorts':(high > USEquityPricing.close.latest * 2) & (sma_20>sma_50),
            'high': high,
            'price': USEquityPricing.close.latest,
            'sma20': sma_20,
            'sma50': sma_50,
            'spy': spy252},
            
            screen=universe)
    return pipe

result = run_pipeline(make_pipeline(), start_date='2015-01-01', end_date='2016-01-01')
result.head()
    